In [1]:
!pip install backtrader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 2.7 MB/s eta 0:00:00


In [23]:
import pandas as pd
import numpy as np
import backtrader as bt
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/cfrm523/Final_project')

<ipython-input-23-bcb74f0062b0>:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
from datetime import datetime

In [17]:
data = pd.read_csv('crypto_mar_april.csv')
data['date'] = pd.to_datetime(data['date'])
data = data.drop('Unnamed: 0', axis=1)

In [18]:
df = data.loc[data['date'] >= datetime(2024,4,1,0,0,0)]
df = df.loc[df['date'] <= datetime(2024,4,2,0,0,0)]
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [36]:
btc = df.loc[df['ticker']=='BTC-USD']
btc = btc.reset_index(drop=True)
btc = btc.set_index('date')
btc.head()

,ticker,open,high,low,close,volume
date,,,,,,
2024-04-01 00:00:00,BTC-USD,71291.50,71313.80,71200.02,71213.11,17.132103
2024-04-01 00:01:00,BTC-USD,71213.11,71284.28,71188.01,71247.73,11.789390
2024-04-01 00:02:00,BTC-USD,71247.69,71263.56,71156.01,71181.64,21.443694
2024-04-01 00:03:00,BTC-USD,71192.70,71254.05,71125.04,71253.92,10.107710
2024-04-01 00:04:00,BTC-USD,71252.56,71280.58,71224.06,71240.05,3.761905


In [79]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy '''
        if dt is None:
            # Access the datetime index from the current line in the data series
            dt = self.datas[0].datetime.datetime(0)

        # Check if dt is still a float (the internal representation for Backtrader), and convert it if needed
        if isinstance(dt, float):
            # Convert backtrader float date to datetime
            dt = bt.num2date(dt)

        # Format datetime object to string
        dt_str = dt.strftime('%Y-%m-%d %H:%M:%S')
        print('%s, %s' % (dt_str, txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close price is: , %.2f' % self.dataclose[0])
        if self.dataclose[0] < self.dataclose[-1]:# current close less than previous close
            if self.dataclose[-1] < self.dataclose[-2]: # previous close less than the previous close
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.buy()

In [80]:
cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy)
btc_bt = bt.feeds.PandasData(dataname=btc)
cerebro.adddata(btc_bt)

In [81]:
cerebro.run()

2024-04-01 00:00:00, Close price is: , 71213.11
2024-04-01 00:01:00, Close price is: , 71247.73
2024-04-01 00:02:00, Close price is: , 71181.64
2024-04-01 00:03:00, Close price is: , 71253.92
2024-04-01 00:04:00, Close price is: , 71240.05
2024-04-01 00:05:00, Close price is: , 71241.19
2024-04-01 00:06:00, Close price is: , 71177.15
2024-04-01 00:07:00, Close price is: , 71185.75
2024-04-01 00:08:00, Close price is: , 71175.89
2024-04-01 00:09:00, Close price is: , 71229.14
2024-04-01 00:10:00, Close price is: , 71155.72
2024-04-01 00:11:00, Close price is: , 71180.69
2024-04-01 00:12:00, Close price is: , 71172.34
2024-04-01 00:13:00, Close price is: , 71192.28
2024-04-01 00:14:00, Close price is: , 71174.36
2024-04-01 00:15:00, Close price is: , 71157.88
2024-04-01 00:15:00, BUY CREATE, 71157.88
2024-04-01 00:16:00, Close price is: , 71161.78
2024-04-01 00:17:00, Close price is: , 71176.15
2024-04-01 00:18:00, Close price is: , 71206.70
2024-04-01 00:19:00, Close price is: , 71222.5